In [ ]:
!pip install opencv-python
!pip install matplotlib

In [1]:
import cv2 as cv
import os
import numpy as np

In [361]:
folder_imagini = os.getcwd() #Tema1
folder_imagini = os.path.join(folder_imagini, 'antrenare') # Tema1/antrenare
continut_folder_imagini = [x for x in os.listdir(folder_imagini) if x.endswith('.txt') or x.endswith('.jpg')]

def show_image(title, image):
    image = cv.resize(image,(0,0), fx=0.25, fy=0.25)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def read_image(image_path):
    #openCv o citeste ca fiind BGR
    img = cv.imread(image_path)
    return img.copy()
    
def apply_hsv_filter(image, l_h, l_s, l_v, u_h, u_s, u_v):
    #BGR in BGR out
    #h -> hue
    #s -> saturation
    #v -> value
    #l -> low
    #h -> high
    copie = image.copy()
    copie = cv.cvtColor(copie, cv.COLOR_BGR2HSV)
    l = np.array([l_h, l_s, l_v])#lower bound pe cele 3 canale
    u = np.array([u_h, u_s, u_v])#upper bound pe cele 3 canale
    mask_table_hsv = cv.inRange(copie, l, u)
    #show_image('masca_hsv', mask_table_hsv)
    rez = cv.bitwise_and(copie, copie, mask=mask_table_hsv)
    rez = cv.cvtColor(rez, cv.COLOR_HSV2BGR)
    return rez

def find_contour_positions(edges):
    show_image('edge_image', edges)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    return top_left, top_right, bottom_left, bottom_right

In [3]:
#edge detection is susceptible to noise in the image

In [386]:
def extrage_tabla_joc(imagine):
    #in IMAGINE out TABLA DE JOC in BGR(Cu tot cu scrisul Mathable)
    original = imagine.copy()
    imagine = apply_hsv_filter(imagine, 14, 0, 0, 255, 255, 255)#primul parametru e bun de la 14 in sus
    show_image('imagine_dupa_filtru_hsv', imagine)

    imagine = cv.cvtColor(imagine, cv.COLOR_BGR2GRAY)
    # image_m_blur = cv.medianBlur(imagine, 5)
    # image_g_blur = cv.GaussianBlur(image_m_blur, (0,0), 18)
    # image_sharpened = cv.addWeighted(image_m_blur, -0.8, image_g_blur, 1.2, 0)
    # image_sharpened = image_m_blur
    # show_image('imagine_dupa_sharpening', image_m_blur)
    
    # _, thresh = cv.threshold(image_sharpened, 40, 255, cv.THRESH_BINARY)
    # kernel = np.ones((6,6), np.uint8)
    # thresh = cv.erode(thresh, kernel)
    # show_image('image_thresholded', thresh)

    # edges = cv.Canny(thresh, 10, 255)

    top_left, top_right, bottom_left, bottom_right = find_contour_positions(imagine)
    
    image_copy = cv.cvtColor(imagine.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    show_image("detected corners",image_copy)

    width = top_right[0] - top_left[0] #18x18, each cell will have 80 pixels
    height = width
    tabla_joc = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
    colutri = np.array([[0,0], [width, 0], [width, height], [0,height]], dtype='float32')
    M = cv.getPerspectiveTransform(tabla_joc, colutri)
    print(imagine.shape)
    result = cv.warpPerspective(original, M, (width, height))
    crop_image_pixels = 85
    return result[crop_image_pixels:result.shape[0] - crop_image_pixels, 
                    crop_image_pixels:result.shape[1] - crop_image_pixels]



In [387]:
def extrage_careu_joc(imagine):
    #in TABLA DE JOC out DOAR CAREUL in BGR(cred)
    show_image('imagine_tabla_joc', imagine)

    imagine = apply_hsv_filter(imagine, 14, 0, 0, 69, 255, 255)
    show_image('imagine_dupa_hsv', imagine)
    
    imagine = cv.cvtColor(imagine, cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(imagine, 5)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0,0), 5)
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    show_image('image_sharpened', image_sharpened)
    _, thresh = cv.threshold(image_sharpened, 0, 255, cv.THRESH_BINARY)

    kernel = np.ones((10,10), np.uint8)
    thresh = cv.erode(thresh, kernel)
    show_image('image_thresholded', thresh)

    edges = cv.Canny(thresh, 200, 400)
    top_left, top_right, bottom_left, bottom_right = find_contour_positions(edges)
    
    image_copy = cv.cvtColor(imagine.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    show_image("detected corners",image_copy)

extrage_careu_joc(tabla)

In [243]:
img = read_image(os.path.join(folder_imagini, 'test.jpg'))
tabla = extrage_tabla_joc(img)

(4080, 3072)
